## 智能体优化 - 反思
****

首先定一个最简单的任务：撰写五段式文章

In [2]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv
import os
# 加载 .env 文件
load_dotenv()

# 创建聊天提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # 系统消息提示词：你是一名文章助手，负责撰写优质的五段式文章。
            # 根据用户的请求生成最佳文章。
            # 如果用户提供了反馈或批评，请根据之前的尝试生成修订版本。
            "你是一名文章助手，负责撰写优质的五段式文章。"
            "根据用户的请求生成最佳文章。"
            "如果用户提供了反馈或批评，请根据之前的尝试生成修订版本。",
        ),
        # 消息占位符，用于动态插入用户和 AI 的消息
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# 配置语言模型
llm = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
)

# 将提示模板与语言模型连接，用于生成内容
generate = prompt | llm


试一下不做反思的效果

In [3]:
essay = ""
request = HumanMessage(
    content="写一篇关于科学用脑的文章，要求内容丰富，语言通俗易懂。"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

科学用脑：让大脑高效运转的五个秘诀

第一段：认识我们的大脑
大脑是人体的指挥中心，重量仅占体重的2%，却消耗着20%的能量。这个由860亿个神经元组成的复杂网络，每时每刻都在处理海量信息。科学研究表明，大脑具有惊人的可塑性，通过科学的方法，我们完全可以提升大脑的工作效率。了解大脑的基本运作原理，是科学用脑的第一步。

第二段：合理作息是基础
大脑就像精密的仪器，需要规律的保养。保证7-8小时的优质睡眠至关重要，因为睡眠时大脑会进行"大扫除"，清除代谢废物。午间20分钟的小憩能显著提升下午的工作效率。此外，遵循自然的昼夜节律，避免熬夜，能让大脑在最佳状态运转。记住，休息不是浪费时间，而是为更高效率的工作充电。

第三段：营养与运动的双重保障
大脑需要优质"燃料"。富含Omega-3脂肪酸的深海鱼、抗氧化能力强的蓝莓、提供持久能量的全谷物都是健脑好选择。同时，适量运动能促进大脑血液循环，刺激脑源性神经营养因子(BDNF)的分泌，这种物质被称为"大脑的肥料"。建议每周进行3-5次、每次30分钟的有氧运动，让身体和大脑一起活力满满。

第四段：科学训练提升脑力
像肌肉一样，大脑也需要针对性训练。尝试学习新技能可以建立新的神经连接，解谜游戏能锻炼逻辑思维，正念冥想则能增强专注力。交替使用不同认知功能，如将语言学习与数学练习穿插进行，可以避免大脑疲劳。重要的是保持适度的挑战性，太简单会无聊，太难则容易放弃。

第五段：情绪管理的艺术
压力和焦虑是大脑效率的"杀手"。长期压力会导致海马体萎缩，影响记忆力。培养积极乐观的心态，通过深呼吸、听音乐等方式调节情绪，能为大脑创造最佳工作环境。社交活动也能刺激大脑，与朋友交流时，多个脑区会协同工作。记住，快乐的大脑才是高效的大脑。

结语：科学用脑不是复杂的学问，而是将健康的生活方式与有针对性的训练相结合。只要我们了解大脑的运作规律，并给予适当的照顾，每个人都能释放大脑的无限潜能。从今天开始，用科学的方法善待我们最宝贵的大脑吧！

构建反思

In [4]:
# 创建反思提示模板
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # 系统消息提示词：你是一名教师，负责对文章提交进行评分。
            # 生成对用户提交的文章的批评意见和建议。
            # 提供详细的建议，包括对文章长度、深度、风格等方面的要求。
            "你是一名教师，负责对文章提交进行评分。"
            "生成对用户提交的文章的批评意见和建议。"
            "提供详细的建议，包括对文章长度、深度、风格等方面的要求。",
        ),
        # 消息占位符，用于动态插入用户和 AI 的消息
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# 将反思提示模板与语言模型连接，用于生成反馈内容
reflect = reflection_prompt | llm

手动进行反思

In [5]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

这篇关于科学用脑的文章整体完成度较高，但仍有一些可以改进的地方。以下是我的详细评阅意见：

1. 文章结构与内容：
- 优点：采用了总分总的结构，五个段落分别阐述了不同方面的科学用脑方法，层次分明。
- 建议：可以增加一些具体的研究数据或案例来增强说服力，比如"午间20分钟小憩"的具体研究来源。

2. 语言表达：
- 优点：语言通俗易懂，使用了恰当的比喻（如"大脑像精密仪器"），专业术语解释到位。
- 建议：部分句子可以更精炼，如第一段最后两句可以合并，避免重复"大脑"一词。

3. 深度与广度：
- 优点：涵盖了睡眠、营养、运动、训练、情绪等多个重要方面。
- 建议：可以增加一小节关于"数字时代如何保护大脑"，讨论信息过载对大脑的影响。

4. 科学性：
- 优点：引用了BDNF等专业概念，显示了研究的深度。
- 建议：专业术语如"海马体"可以加括号简单解释，方便普通读者理解。

5. 实用性：
- 优点：提供了具体可操作的建议（如运动频率、午休时长）。
- 建议：可以增加一些"小贴士"式的实用技巧，比如"5分钟办公室健脑操"。

6. 改进建议：
- 增加过渡句使段落衔接更自然
- 部分数据可以标注来源增强可信度
- 结尾可以更富有感染力，加入号召性语言
- 考虑添加1-2个图表来直观展示关键信息

7. 格式建议：
- 为每个小标题添加编号
- 重点数据可以用加粗突出
- 过长的段落可以适当拆分

评分：85/100
这是一篇质量上乘的科普文章，只需在细节处稍加打磨，就能成为一篇优秀的科学用脑指南。作者展现了对主题的深入理解，且能够用通俗语言传达科学知识，这种能力值得肯定。

建议作者：
1. 补充2-3个权威研究引用
2. 增加一个"常见误区"的小节
3. 优化段落间的过渡
4. 考虑添加互动元素（如自测题）

期待看到修改后的版本！

定义graph

In [6]:
from typing import Annotated, List, Sequence
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict


class State(TypedDict):
    # 定义状态类型，其中消息列表使用 `add_messages` 进行注解
    messages: Annotated[list, add_messages]


# 生成节点：处理生成的逻辑
async def generation_node(state: State) -> State:
    # 调用生成器的异步方法，根据当前状态的消息生成新的消息
    return {"messages": [await generate.ainvoke(state["messages"])]}


# 反思节点：处理反思和反馈的逻辑
async def reflection_node(state: State) -> State:
    # 需要调整的其他消息
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # 第一条消息是用户的原始请求，我们在所有节点中保持不变
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    # 调用反思器的异步方法，处理调整后的消息
    res = await reflect.ainvoke(translated)
    # 将反思的输出视为生成器的用户反馈
    return {"messages": [HumanMessage(content=res.content)]}


# 创建状态图构建器
builder = StateGraph(State)
# 添加生成节点
builder.add_node("generate", generation_node)
# 添加反思节点
builder.add_node("reflect", reflection_node)
# 添加从起始节点到生成节点的边
builder.add_edge(START, "generate")


# 定义是否继续的条件函数
def should_continue(state: State):
    if len(state["messages"]) > 6:
        # 如果消息数量超过 6，则结束（相当于 3 次迭代后结束）
        return END
    # 否则继续到反思节点
    return "reflect"


# 添加条件边，根据条件决定下一步是结束还是进入反思节点
builder.add_conditional_edges("generate", should_continue)
# 添加从反思节点到生成节点的边
builder.add_edge("reflect", "generate")
# 使用内存保存器作为检查点
memory = MemorySaver()
# 编译状态图，使用检查点保存器
graph = builder.compile(checkpointer=memory)

In [7]:
config = {"configurable": {"thread_id": "1"}}

In [8]:
async for event in graph.astream(
    {
        "messages": [
            HumanMessage(
                content="写一篇关于科学用脑的文章，要求内容丰富，语言通俗易懂。"
            )
        ],
    },
    config,
):
    print(event)
    print("---")

{'generate': {'messages': [AIMessage(content='科学用脑：让大脑高效运转的五个秘诀\n\n第一段：认识大脑的工作机制\n我们的大脑就像一台24小时运转的超级计算机，重约1.4公斤却消耗着人体20%的能量。科学研究表明，大脑由860亿个神经元组成，通过突触连接形成复杂的神经网络。与电脑不同，大脑具有"用进废退"的特性，越使用越灵活。了解这些基本知识，是科学用脑的第一步。\n\n第二段：合理分配脑力时间\n大脑并非永动机，它需要科学的休息节奏。研究表明，成人专注力通常只能维持45-90分钟。建议采用"番茄工作法"：专注25分钟后休息5分钟。此外，大脑在早晨和睡前两小时记忆效率最高，这段时间最适合学习新知识。午间小憩20分钟能让下午的工作效率提升34%。\n\n第三段：营养与运动的双重保障\n科学用脑离不开健康饮食。Omega-3脂肪酸（深海鱼、坚果）、抗氧化剂（蓝莓、黑巧克力）和B族维生素（全谷物）都是大脑的"优质燃料"。同时，适量运动能促进大脑分泌BDNF（脑源性神经营养因子），这个"大脑肥料"能刺激新神经元生长。建议每周进行3-5次有氧运动，每次30分钟。\n\n第四段：多元化的用脑方式\n大脑喜欢新鲜感。交替进行不同类型的脑力活动（如逻辑推理后切换艺术创作）能激活不同脑区，避免疲劳。学习新技能（如外语、乐器）可以建立新的神经连接。睡眠时大脑会整理记忆，因此保证7-8小时优质睡眠至关重要。偶尔玩些益智游戏也是不错的"大脑体操"。\n\n第五段：情绪管理的艺术\n压力会抑制海马体（记忆中枢）的神经元生长。正念冥想、深呼吸等放松技巧能降低压力激素水平。保持好奇心和学习热情会刺激多巴胺分泌，让学习变得愉悦。建议建立"成长型思维"，把挑战视为锻炼大脑的机会。记住，快乐的心情才是最好的"脑力增强剂"。\n\n科学用脑不是天赋，而是可以培养的习惯。通过理解大脑特性、合理安排时间、注重身心健康、多样化训练和情绪管理，每个人都能开发出更大的脑力潜能。让我们从今天开始，用科学的方法善待这个最精密的"生物计算机"吧！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 486, 'prompt_tokens

查看详细过程

In [9]:
state = graph.get_state(config)
ChatPromptTemplate.from_messages(state.values["messages"]).pretty_print()

================================ Human Message =================================

写一篇关于科学用脑的文章，要求内容丰富，语言通俗易懂。

================================== Ai Message ==================================

科学用脑：让大脑高效运转的五个秘诀

第一段：认识大脑的工作机制
我们的大脑就像一台24小时运转的超级计算机，重约1.4公斤却消耗着人体20%的能量。科学研究表明，大脑由860亿个神经元组成，通过突触连接形成复杂的神经网络。与电脑不同，大脑具有"用进废退"的特性，越使用越灵活。了解这些基本知识，是科学用脑的第一步。

第二段：合理分配脑力时间
大脑并非永动机，它需要科学的休息节奏。研究表明，成人专注力通常只能维持45-90分钟。建议采用"番茄工作法"：专注25分钟后休息5分钟。此外，大脑在早晨和睡前两小时记忆效率最高，这段时间最适合学习新知识。午间小憩20分钟能让下午的工作效率提升34%。

第三段：营养与运动的双重保障
科学用脑离不开健康饮食。Omega-3脂肪酸（深海鱼、坚果）、抗氧化剂（蓝莓、黑巧克力）和B族维生素（全谷物）都是大脑的"优质燃料"。同时，适量运动能促进大脑分泌BDNF（脑源性神经营养因子），这个"大脑肥料"能刺激新神经元生长。建议每周进行3-5次有氧运动，每次30分钟。

第四段：多元化的用脑方式
大脑喜欢新鲜感。交替进行不同类型的脑力活动（如逻辑推理后切换艺术创作）能激活不同脑区，避免疲劳。学习新技能（如外语、乐器）可以建立新的神经连接。睡眠时大脑会整理记忆，因此保证7-8小时优质睡眠至关重要。偶尔玩些益智游戏也是不错的"大脑体操"。

第五段：情绪管理的艺术
压力会抑制海马体（记忆中枢）的神经元生长。正念冥想、深呼吸等放松技巧能降低压力激素水平。保持好奇心和学习热情会刺激多巴胺分泌，让学习变得愉悦。建议建立"成长型思维"，把挑战视为锻炼大脑的机会。记住，快乐的心情才是最好的"脑力增强剂"。

科学用脑不是天赋，而是可以培养的习惯。通过理解大脑特性、合理安排时间、注重身心健康、多样化训练和情绪管理，每个人都能开发出更大的脑力潜能。让我们从今天开始，用科学的方法善待